In [1]:
import sys
sys.path.append("../bus_service_increase")

In [2]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import altair as alt
import calitp_data_analysis.magics
import pandas as pd

from IPython.display import display, HTML

from bus_service_utils import chart_utils
from calitp_data_analysis import calitp_color_palette as cp
from update_vars import GCS_FILE_PATH, PUBLIC_FILENAME, YEAR, MONTH

#alt.renderers.enable("html")
alt.data_transformers.enable('default', max_rows=None)

In [22]:
# parameters cell for local
rtpa = "Metropolitan Transportation Commission"

In [4]:
%%capture_parameters
rtpa

{"rtpa": "Riverside County Transportation Commission"}


# {rtpa}
## Monthly Ridership Trends

**Download data from our public [folder](https://console.cloud.google.com/storage/browser/calitp-publish-data-analysis)** by navigating to `ntd_monthly_ridership` and selecting a file.

In [5]:
URL = ("https://console.cloud.google.com/storage/"
       "browser/calitp-publish-data-analysis"
      )

display(
    HTML(
        f"""
        <a href={URL}>
        Download the latest month of data: {PUBLIC_FILENAME}</a>
        """
    )
)


In [23]:
MIN_YEAR = 2018

#updated to filter for period_year greater than 2018
df = pd.read_parquet(
    f"{GCS_FILE_PATH}ca_monthly_ridership_{YEAR}_{MONTH}.parquet",
    filters = [[("RTPA", "==", rtpa), ("period_year", ">=", MIN_YEAR)]]
).drop(
    #updated columns names to match new df
    columns = ["mode", "tos"]
).rename(columns = {"Mode_full": "Mode", "TOS_full": "TOS"})

In [8]:
def sum_by_group(
    df: pd.DataFrame,
    group_cols: list) -> pd.DataFrame:
    """
    since data is now long, this replaces old sum_by_group, make_long and assemble_long_df functions.
    """
    from monthly_ridership_by_rtpa import get_percent_change
    grouped_df = df.groupby(group_cols+
                             ['period_year',
                             'period_month',
                             'period_year_month']
                           ).agg({
        "upt":"sum",
        "previous_y_m_upt":"sum",
        "change_1yr":"sum"
    }
    ).reset_index()
    
    #get %change back
    grouped_df = get_percent_change(grouped_df)
    
    #decimal to whole number
    grouped_df["pct_change_1yr"] = grouped_df["pct_change_1yr"]*100
    
    return grouped_df

In [24]:
agency_cols = ["NTD ID", "Agency", "RTPA"]
mode_cols = ["Mode", "RTPA"]
tos_cols = ["TOS", "RTPA"]

by_agency = sum_by_group(df, agency_cols)
by_mode = sum_by_group(df, mode_cols)
by_tos = sum_by_group(df, tos_cols)

In [27]:
def remove_zero_upt_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    takes the by_agency_long df, filters for rows with zero UPT AND zero change_1yr. 
    then removes these rows from the by_agency_long df.
    resulting df should only contain rows with change in UPT not due to (0 UPT - 0 UPT) and net zero change in UPT
    zero UPT and zero change_1yr occurs when (0 upt - 0upt = 0change), dont need to show this
    but a net zero change (100upt - 100upt = 0 change), should be shown
    """
    
    #df of rows with zero UPT for 2 years
    zero_UPT_2years = df[(df["upt"] == 0) & (df["change_1yr"] == 0)]
    
    merge = df.merge(zero_UPT_2years, how="left", indicator=True)
    no_zero_UPT_rows = merge[merge["_merge"] == "left_only"].drop(columns=["_merge"]).dropna(subset="change_1yr")
    
    return no_zero_UPT_rows
    

In [ ]:
agency_cols = ["ntd_id", "agency", "RTPA"]
mode_cols = ["Mode", "RTPA"]
tos_cols = ["TOS", "RTPA"]

by_agency_long = sum_by_group(df, agency_cols)
by_mode_long = sum_by_group(df, mode_cols)
by_tos_long = sum_by_group(df, tos_cols)

# remove zero UPT rows 
by_agency_long_no_zero_upt = remove_zero_upt_rows(by_agency_long)

In [13]:
LABELING_DICT = {
    "upt": "Unlinked Passenger Trips",
    "change_1yr": "Change in Unlinked Passenger Trips from Prior Year",
    "TOS": "Type of Service",
    "year_month": "Date"
}

def labeling(word: str) -> str:
    return chart_utils.labeling(word, LABELING_DICT)

WIDTH = 300
HEIGHT = 150

In [14]:
def make_line_chart(
    df: pd.DataFrame, 
    y_col: str,
    color_col: str,
) -> alt.Chart:
    df = df[df[y_col] > 0].dropna(subset = y_col)

    x_label = [i for i in df.period_year_month.unique() if 
               any(substring in i for substring in 
                   ["-01", "-06"])
              ]    
    chart = (alt.Chart(df)
         .mark_line()
         .encode(
             x = alt.X("period_year_month:O", 
                       axis=alt.Axis(values = x_label), 
                       title = "Date"
                      ),
             y = alt.Y(y_col, title = labeling(y_col)),
             color = alt.Color(color_col, title = "", 
                               scale = alt.Scale(
                                   range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                                   cp.CALITP_CATEGORY_BOLD_COLORS
                               )),
             tooltip = ["period_year_month", y_col, color_col, "RTPA"]
         ).properties(width = WIDTH, height = HEIGHT)
         .facet(color_col, columns=2, title = "")
         .resolve_scale(y="independent")
    ).properties(
        title = f"{labeling(y_col)} by {labeling(color_col)}"
    ).interactive()
        
    return chart

In [15]:
def make_bar_chart(
    df: pd.DataFrame, 
    y_col: str,
    color_col: str,
) -> alt.Chart:
    
    def short_label(word):
        shorten_dict = {
            "change_1yr": "Change",
            "pct_change_1yr":"Change",
        }
        return shorten_dict[word]
    
    # For change column, we are missing everything prior to 2023 
    #df = df.dropna(subset = y_col)
    
    #need flag for y_col >,<, 0, missing?
    #count function to how many agencies fall in those categories, then look at those agencies
    #present table
    
    x_label = [i for i in df.period_year_month.unique() if 
               any(substring in i for substring in 
                   ["-01", "-03", "-06", "-09"])
              ]
    
    chart = (alt.Chart(df)
         .mark_bar()
         .encode(
             x = alt.X("period_year_month:O", 
                       axis=alt.Axis(values = x_label), 
                       title = "Date"
                      ),
             y = alt.Y(y_col, title = short_label(y_col)),
             color = alt.Color(color_col, title = "", 
                              scale = alt.Scale(
                                   range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                                   cp.CALITP_CATEGORY_BOLD_COLORS
                              )),
             tooltip = ["period_year_month", y_col, color_col, "RTPA"]
         ).properties(width = WIDTH, height = HEIGHT)
         .facet(color_col, columns=2, title = "")
         .resolve_scale(x="shared", 
                        y="independent")
    ).properties(
        title = f"{labeling(y_col)} by {labeling(color_col)}"
    ).interactive()
    
    return chart

### Transit Agency

In [16]:
make_line_chart(by_agency_long, y_col = "upt", color_col = "agency")

alt.FacetChart(...)

Change in Unlinked Passenger Trips from the prior year. For example, July 2023's change would be the change in July 2023's reported values against July 2022's reported values.

In [26]:
make_bar_chart(by_agency_long_no_zero_upt, y_col = "change_1yr", color_col = "Agency")

alt.FacetChart(...)

### Transit Mode

In [18]:
make_line_chart(by_mode_long, y_col = "upt", color_col = "Mode")

alt.FacetChart(...)

In [19]:
make_bar_chart(by_mode_long, y_col = "change_1yr", color_col = "Mode")

alt.FacetChart(...)

### Type of Service

In [20]:
make_line_chart(by_tos_long, y_col = "upt", color_col = "TOS")

alt.FacetChart(...)

In [28]:
make_bar_chart(by_tos_long, y_col = "change_1yr", color_col = "TOS")

alt.FacetChart(...)